In [ ]:
from data_utils import DataLoader
from spectral_mix import SpectralMix
import time
import pandas as pd

In [ ]:
dl = DataLoader()

dataset = dl.load_dataset('acm')
d = 9

num_nodes, _, num_rels = dataset['adjacency_matrix'].shape
_, num_attr = dataset['attribute_matrix'].shape

# runtime test for increasing n
runtime_fixed = pd.DataFrame(columns=['n', 'runtime']) 
for n in range(500, num_nodes, 500):
    # take subset of adjacency matrix and attribute matrix
    adjacency_matrix = dataset['adjacency_matrix'][:n, :n, :]
    attribute_matrix = dataset['attribute_matrix'][:n, :]

    sm = SpectralMix(d=d)
    begin = time.time()
    sm.fit(adjacency_matrix=adjacency_matrix, attribute_matrix=attribute_matrix, run_clustering=False)
    runtime = time.time() - begin

    result = []
    result.append(n)
    result.append(runtime)
    runtime_fixed.loc[len(runtime_fixed)] = result

    runtime_fixed.to_csv('test_results/runtime_fixed.csv')

# runtime test for increasing n
runtime_var_r = pd.DataFrame(columns=['n', 'runtime']) 
for r in range(num_rels):
    # take subset of adjacency matrix and attribute matrix
    adjacency_matrix = dataset['adjacency_matrix'][:, :, :r]
    attribute_matrix = dataset['attribute_matrix']

    sm = SpectralMix(d=d)
    begin = time.time()
    sm.fit(adjacency_matrix=adjacency_matrix, attribute_matrix=attribute_matrix, run_clustering=False)
    runtime = time.time() - begin

    result = []
    result.append(n)
    result.append(runtime)
    runtime_var_r.loc[len(runtime_var_r)] = result

    runtime_var_r.to_csv('test_results/runtime_var_r.csv')


# runtime test for increasing n
runtime_var_a = pd.DataFrame(columns=['n', 'runtime']) 
for a in range(num_attr):
    # take subset of adjacency matrix and attribute matrix
    adjacency_matrix = dataset['adjacency_matrix']
    attribute_matrix = dataset['attribute_matrix'][:, :a]

    sm = SpectralMix(d=d)
    begin = time.time()
    sm.fit(adjacency_matrix=adjacency_matrix, attribute_matrix=attribute_matrix, run_clustering=False)
    runtime = time.time() - begin

    result = []
    result.append(n)
    result.append(runtime)
    runtime_var_a.loc[len(runtime_var_a)] = result

    runtime_var_a.to_csv('test_results/runtime_var_r.csv')